In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Source #Partner

### Links: 
___

In [ ]:
query_text = """--sql
    CREATE TABLE db1.users_st_partner
    (
        `report_date` Date,
        `partner_uuid`  String,
        `last_logging` DateTime
    )
    ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/users_st_partner/year=*/month=*/*.csv','CSVWithNames')
    """

ch.query_run(query_text)

In [ ]:
query_text = """--sql
    CREATE TABLE db1.users_st_partner_ch
    (
        `report_date` Date,
        `partner_uuid`  String,
        `last_logging` DateTime
    )
    ENGINE = MergeTree()
    ORDER BY partner_uuid
    """

ch.query_run(query_text)


In [ ]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.users_st_partner_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.users_st_partner_ch AS
    SELECT
        *
    FROM db1.users_st_partner
    """

ch.query_run(query_text)

___
## Tools
___
### query

In [ ]:
query_text = """--sql
    SELECT
        *
    FROM db1.users_st_partner_ch
    limit 10
    """

ch.query_run(query_text)

### Drop ch

In [ ]:
query_text = """
    DROP TABLE db1.users_st_partner_ch
    """
ch.query_run(query_text)

### Drop mv

In [2]:
query_text = """
    DROP TABLE db1.users_st_partner_mv
    """
ch.query_run(query_text)

,read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
0,0,0,0,0,0,0,0,1189848,f26f06ab-bf59-4c4c-b0be-a687ff3d2316
